# ELEC 475 Lab 4 - CLIP Visualization Generator

This notebook generates all required visualizations for Lab 4 Section 2.4:
1. Text→Image retrieval (including 'sport' and 'a dog playing')
2. Zero-shot image classification
3. Comparison between base CLIP and fine-tuned models

**Features:**
- Clones Lab4 code from GitHub
- Downloads ONLY val2014 images (~6GB, not entire 20+GB dataset)
- Generates visualizations for both base and fine-tuned models
- Saves outputs to Google Drive

## Setup: Mount Google Drive and Install Dependencies

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install -q transformers torch torchvision tqdm pillow matplotlib

## Clone Lab4 Code from GitHub

In [ ]:
import os

# Clone repository
if not os.path.exists('475_ML-CV_Labs'):
    print("Cloning repository from GitHub...")
    !git clone https://github.com/Jcub05/475_ML-CV_Labs.git
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Navigate to Lab4 directory
os.chdir('/content/475_ML-CV_Labs/Lab4')
print(f"✓ Current directory: {os.getcwd()}")

# Verify required files exist
required_files = ['model.py', 'visualize.py', 'metrics.py']
for f in required_files:
    if os.path.exists(f):
        print(f"✓ {f} found")
    else:
        print(f"✗ {f} missing!")

## Download ONLY val2014 Images (~6GB)

This downloads only the validation images, not the entire 20+GB dataset.

In [ ]:
import urllib.request
import zipfile
from pathlib import Path

# Setup directories
data_dir = Path('/content/coco_data')
data_dir.mkdir(exist_ok=True)

val_zip_path = data_dir / 'val2014.zip'
val_dir = data_dir / 'val2014'

# Download val2014 images if not already downloaded
if not val_dir.exists() or len(list(val_dir.glob('*.jpg'))) == 0:
    print("Downloading COCO val2014 images (~6GB)...")
    print("This will take 5-10 minutes depending on connection speed.")
    
    # Download from official COCO website
    val_url = 'http://images.cocodataset.org/zips/val2014.zip'
    
    # Download with progress
    def download_progress(block_num, block_size, total_size):
        downloaded = block_num * block_size
        percent = min(100, downloaded * 100 / total_size)
        print(f'\rDownloading: {percent:.1f}% ({downloaded / 1e9:.2f} GB / {total_size / 1e9:.2f} GB)', end='')
    
    urllib.request.urlretrieve(val_url, val_zip_path, download_progress)
    print("\n✓ Download complete")
    
    # Extract
    print("\nExtracting val2014.zip...")
    with zipfile.ZipFile(val_zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    print("✓ Extraction complete")
    
    # Remove zip file to save space
    val_zip_path.unlink()
    print("✓ Removed zip file to save space")
else:
    print("✓ val2014 images already downloaded")

# Verify
num_val_images = len(list(val_dir.glob('*.jpg')))
print(f"\n✓ Validation images found: {num_val_images}")
print(f"✓ Image directory: {val_dir}")

## Upload Your Trained Model Checkpoint

Upload `best_model.pth` or your checkpoint file from Google Drive or local machine.

In [ ]:
from google.colab import files

# Option 1: Upload from local machine
print("Upload your trained model checkpoint (e.g., best_model.pth)")
uploaded = files.upload()
model_checkpoint_path = list(uploaded.keys())[0]
print(f"✓ Model checkpoint: {model_checkpoint_path}")

# Option 2: If checkpoint is in Google Drive, uncomment and modify:
# model_checkpoint_path = '/content/drive/MyDrive/datasets_Lab4/Lab4_checkpoints/best_model.pth'

## Visualization Code

Complete visualization generation script with base and fine-tuned model support.

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

from model import CLIPFineTuneModel
from visualize import (
    visualize_text_to_image_retrieval,
    zero_shot_classification,
    create_retrieval_grid
)

# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
output_dir = Path('/content/Visualizations')
output_dir.mkdir(exist_ok=True)
num_images = 1000  # Number of val images to use

print(f"Device: {device}")
print(f"Output directory: {output_dir}")

In [ ]:
def get_clip_transform():
    """Get CLIP preprocessing transform."""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.48145466, 0.4578275, 0.40821073),
            std=(0.26862954, 0.26130258, 0.27577711)
        )
    ])


def load_base_clip_model(device):
    """Load the base OpenAI CLIP model."""
    print("Loading base OpenAI CLIP model...")
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    model.eval()
    print("✓ Base CLIP model loaded")
    return model


def load_finetuned_model(model_path, device):
    """Load fine-tuned CLIP model."""
    print(f"Loading fine-tuned model from {model_path}...")
    
    model = CLIPFineTuneModel(
        embed_dim=512,
        pretrained_resnet=True,
        clip_model_name="openai/clip-vit-base-patch32",
        freeze_text_encoder=True
    ).to(device)
    
    checkpoint = torch.load(model_path, map_location=device)
    state_dict = checkpoint.get('model_state_dict', checkpoint)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    
    print("✓ Fine-tuned model loaded")
    return model


def precompute_image_embeddings_finetuned(model, image_paths, transform, device, batch_size=32):
    """Precompute embeddings using fine-tuned model."""
    print(f"\nPrecomputing embeddings for {len(image_paths)} images (fine-tuned)...")
    
    all_embeds = []
    model.eval()
    
    with torch.no_grad():
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]
            
            images = []
            for path in batch_paths:
                img = Image.open(path).convert('RGB')
                images.append(transform(img))
            
            images = torch.stack(images).to(device)
            embeds = model.encode_image(images).cpu()
            all_embeds.append(embeds)
            
            if (i // batch_size + 1) % 10 == 0:
                print(f"  Processed {i+len(batch_paths)}/{len(image_paths)}")
    
    all_embeds = torch.cat(all_embeds, dim=0)
    print(f"✓ Embeddings computed: {all_embeds.shape}")
    return all_embeds


def precompute_image_embeddings_base(model, image_paths, processor, device, batch_size=32):
    """Precompute embeddings using base CLIP model."""
    print(f"\nPrecomputing embeddings for {len(image_paths)} images (base CLIP)...")
    
    all_embeds = []
    model.eval()
    
    with torch.no_grad():
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]
            
            images = []
            for path in batch_paths:
                img = Image.open(path).convert('RGB')
                images.append(img)
            
            inputs = processor(images=images, return_tensors="pt").to(device)
            embeds = model.get_image_features(**inputs)
            embeds = embeds / embeds.norm(dim=-1, keepdim=True)
            all_embeds.append(embeds.cpu())
            
            if (i // batch_size + 1) % 10 == 0:
                print(f"  Processed {i+len(batch_paths)}/{len(image_paths)}")
    
    all_embeds = torch.cat(all_embeds, dim=0)
    print(f"✓ Embeddings computed: {all_embeds.shape}")
    return all_embeds


class BaseCLIPWrapper:
    """Wrapper for base CLIP to match fine-tuned model interface."""
    def __init__(self, clip_model, processor):
        self.clip_model = clip_model
        self.processor = processor
        
    def eval(self):
        self.clip_model.eval()
        return self
    
    def encode_text(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.clip_model.get_text_features(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            outputs = outputs / outputs.norm(dim=-1, keepdim=True)
        return outputs
    
    def encode_image(self, images):
        with torch.no_grad():
            outputs = self.clip_model.vision_model(pixel_values=images).pooler_output
            outputs = self.clip_model.visual_projection(outputs)
            outputs = outputs / outputs.norm(dim=-1, keepdim=True)
        return outputs


def generate_visualizations_for_model(model, model_name, image_paths, image_embeds, 
                                     processor, transform, device, output_dir):
    """Generate all visualizations for a model."""
    
    print("\n" + "="*70)
    print(f"Generating visualizations for: {model_name}")
    print("="*70)
    
    model_output_dir = output_dir / model_name
    model_output_dir.mkdir(parents=True, exist_ok=True)
    
    # Text→Image Retrieval
    print("\nREQUIREMENT 1: Text→Image Retrieval")
    print("-" * 70)
    
    text_queries = [
        "sport",
        "a dog playing",
        "a person eating",
        "a beautiful sunset",
        "a cat on a couch"
    ]
    
    for query in text_queries:
        print(f"  Query: '{query}'")
        save_path = model_output_dir / f"text2img_{query.replace(' ', '_')}.png"
        
        visualize_text_to_image_retrieval(
            query_text=query,
            model=model,
            image_paths=image_paths,
            image_embeds=image_embeds,
            clip_processor=processor,
            device=device,
            top_k=5,
            save_path=save_path
        )
    
    # Grid view
    print(f"  Creating retrieval grid...")
    grid_path = model_output_dir / "text2img_grid.png"
    create_retrieval_grid(
        queries=text_queries[:4],
        model=model,
        image_paths=image_paths,
        image_embeds=image_embeds,
        clip_processor=processor,
        device=device,
        images_per_query=5,
        save_path=grid_path
    )
    
    # Zero-Shot Classification
    print("\nREQUIREMENT 2: Zero-Shot Image Classification")
    print("-" * 70)
    
    class_labels = ['a person', 'an animal', 'a landscape']
    num_examples = 5
    sample_images = image_paths[:num_examples]
    
    for idx, img_path in enumerate(sample_images):
        print(f"  Classifying image {idx+1}/{num_examples}: {img_path.name}")
        save_path = model_output_dir / f"classification_example_{idx+1}.png"
        
        predicted_class, confidence = zero_shot_classification(
            query_image_path=img_path,
            class_labels=class_labels,
            model=model,
            clip_processor=processor,
            transform=transform,
            device=device,
            save_path=save_path
        )
        
        print(f"    → Predicted: {predicted_class} ({confidence*100:.1f}%)")
    
    print(f"\n✓ All visualizations for {model_name} saved to: {model_output_dir}")


print("✓ All functions defined")

## Load Processor and Get Image Paths

In [ ]:
# Load CLIP processor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
transform = get_clip_transform()

# Get validation image paths
all_image_paths = sorted(list(val_dir.glob("*.jpg")))[:num_images]
print(f"\n✓ Using {len(all_image_paths)} validation images")

## Generate Visualizations for Base CLIP Model

In [ ]:
print("="*70)
print("PART 1: BASE CLIP MODEL (OpenAI Frozen)")
print("="*70)

base_clip = load_base_clip_model(device)
base_clip_wrapped = BaseCLIPWrapper(base_clip, processor)

# Precompute embeddings
base_image_embeds = precompute_image_embeddings_base(
    base_clip, all_image_paths, processor, device
)

# Generate visualizations
generate_visualizations_for_model(
    model=base_clip_wrapped,
    model_name="base_model",
    image_paths=all_image_paths,
    image_embeds=base_image_embeds,
    processor=processor,
    transform=transform,
    device=device,
    output_dir=output_dir
)

# Clean up
del base_clip, base_clip_wrapped, base_image_embeds
torch.cuda.empty_cache()

print("\n✅ Base model visualizations complete!")

## Generate Visualizations for Fine-Tuned Model

In [ ]:
print("="*70)
print("PART 2: FINE-TUNED MODEL (Your Trained Model)")
print("="*70)

finetuned_model = load_finetuned_model(model_checkpoint_path, device)

# Precompute embeddings
finetuned_image_embeds = precompute_image_embeddings_finetuned(
    finetuned_model, all_image_paths, transform, device
)

# Generate visualizations
generate_visualizations_for_model(
    model=finetuned_model,
    model_name="finetuned_model",
    image_paths=all_image_paths,
    image_embeds=finetuned_image_embeds,
    processor=processor,
    transform=transform,
    device=device,
    output_dir=output_dir
)

# Clean up
del finetuned_model, finetuned_image_embeds
torch.cuda.empty_cache()

print("\n✅ Fine-tuned model visualizations complete!")

## Summary and Download Results

In [ ]:
print("\n" + "="*70)
print("✅ VISUALIZATION GENERATION COMPLETE")
print("="*70)
print(f"\n📁 Output directory: {output_dir}")
print(f"\nGenerated visualizations:")

for subdir in output_dir.iterdir():
    if subdir.is_dir():
        num_files = len(list(subdir.glob("*.png")))
        print(f"\n  {subdir.name}/ ({num_files} images)")
        for file_path in sorted(subdir.glob("*.png"))[:5]:
            size_kb = file_path.stat().st_size / 1024
            print(f"    • {file_path.name} ({size_kb:.1f} KB)")
        if num_files > 5:
            print(f"    ... and {num_files - 5} more")

print("\n" + "="*70)
print("Lab 4 Section 2.4 Requirements:")
print("  ✓ Text→Image retrieval (including 'sport' and 'a dog playing')")
print("  ✓ Image classification with ['a person', 'an animal', 'a landscape']")
print("  ✓ Comparison between base and fine-tuned models")
print("="*70)

In [ ]:
# Archive visualizations for download
import shutil

archive_name = '/content/Lab4_Visualizations'
shutil.make_archive(archive_name, 'zip', output_dir)
print(f"\n✓ Created archive: {archive_name}.zip")

# Download the archive
files.download(f'{archive_name}.zip')
print("\n✓ Download started! Check your browser downloads.")

## Optional: Copy to Google Drive

In [ ]:
# Copy visualizations to Google Drive
import shutil

drive_output_dir = '/content/drive/MyDrive/Lab4_Visualizations'

if os.path.exists(drive_output_dir):
    shutil.rmtree(drive_output_dir)

shutil.copytree(output_dir, drive_output_dir)
print(f"✓ Visualizations copied to Google Drive: {drive_output_dir}")